In [ ]:
# magic
%matplotlib inline

# imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
# from IPython.display import SVG, display # svg images
from shapely.geometry import *
from shapely.affinity import *
from shapely.prepared import prep
import graphviz as gv
import random as rand
from math import *
from itertools import permutations
import importlib

# my libraries
from mcr import *
from graph import *
import networkx as nx

# make figures large enough
from pylab import rcParams
rcParams['figure.figsize'] = (10, 10)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

In [ ]:
# Create an empty square
square = MCR()

In [ ]:
# Add a couple of shapes to the square
shape1 = Polygon([(0.2,0.2), (0.2, 0.7), (0.7,0.7), (0.7,0.2)])
shape2 = Polygon([(0.5,0.15), (1.0,0.15), (.75, .87)])
shape3 = Polygon([(0.0, 0.2), (0.0, 0.5), (0.3, 0.5), (0.3,0.2)])

square.add_obstacle(shape1)
square.add_obstacle(shape2)
square.add_obstacle(shape3)

# change its color?! Why not?
shape1.facecolor = 'pink'

# Add a start and goal
square.start = Point(0.05, 0.05)
square.goal  = Point(0.95, 0.95)

In [ ]:
p = Point(1,3)

In [ ]:
p.x

In [ ]:
# Display the MCR
square.plot_obstacles(labels=True)

In [ ]:
square.create_graph()

In [ ]:
square.node_pos

In [ ]:
square.show(labels=True)

In [ ]:
shapes = MCR(svg='shapes.svg')

In [ ]:
shapes.plot_obstacles()

In [ ]:
shapes.create_graph(labels=True)

In [ ]:
shapes.show()

In [ ]:
# The above is equivalent to:
shapes.plot_obstacles()
shapes.plot_graph(labels=False)
shapes.setup_axes()
plt.show()

In [ ]:
straws = MCR(svg='straws.svg')

In [ ]:
straws.plot_obstacles()

In [ ]:
straws.plot_overlapped_obstacles()

In [ ]:
straws.create_graph()

In [ ]:
straws.show()

In [ ]:
f = random_MCR(30)
f.plot_obstacles(labels=True)

In [ ]:
f.show(labels=False)

In [ ]:
### highlight overlappings of three or more obstacles

# find the overlappings and mark them
triples = [o_o for o_o in f.overlapped_obstacles if len(o_o.label.split(',')) >= 3]
for o_o in triples:
    o_o.facecolor = 'red'

# first draw everything
f.plot_obstacles()

MCR.plot_shapes(triples)

# show the graph as well
f.plot_graph(labels=False)
f.setup_axes()
plt.show()

In [ ]:
### highlight obstacles in the direct path from start to goal

# first draw everything
f.plot_obstacles()

# created the LineString and draw it
path = LineString([Point(f.start), Point(f.goal)])
MCR.plot_linestrings([path])

# find the crossed obstacles and show them
crossed = [o_o for o_o in f.overlapped_obstacles if path.crosses(o_o)]
for o_o in crossed:
    o_o.facecolor = 'yellow'
MCR.plot_shapes(crossed)

# show the graph as well
f.plot_graph(labels=False)
f.setup_axes()
plt.show()

In [ ]:
f = random_MCR(20)
f.show()

In [ ]:
G=nx.dodecahedral_graph()
nx.draw_networkx(G)